In [4]:
##
##2021.03.27 #백재원
##

import pandas as pd #campus_tel_20210310
import numpy as np
#import matplotlib
from matplotlib import font_manager

font_location  = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_location).get_name()

def SumData(Data):
    rows = int(Data.shape[1] / 2)
    name = '성명'
    number = '번호'
    TotalData = pd.DataFrame({'성명' : Data[name], '번호' : Data[number]})
    for i in np.arange(1,rows):
        name = name + '.' + str(i)
        number = number + '.' + str(i)
        Temp = pd.DataFrame({'성명' : Data[name], '번호' : Data[number]})
        TotalData = TotalData.append(Temp)
        name = '성명'
        number = '번호'
    return TotalData

Data = pd.read_excel("C:/Temp/campus_tel_20210310.xlsx",  skiprows=2)
#print(Data)
TotalData = SumData(Data)

#TotalData.filter(like='총장' , axis=0).head()
#print(TotalData[TotalData['성명'] == '이장균']['번호'])

TotalData.head(10)

#

,성명,번호
0,총장실,NaN
1,총장 진인주,2000
2,(직통),865-6991
3,(비서) 전시은,2001
4,FAX,868-3408
5,교무처,NaN
6,처장 김용진,2002
7,부처장 김재호,2006
8,교무팀 (본-106),NaN
9,팀장 이형배,2030
